In [ ]:
!pip install langchain
!pip install openai

In [ ]:
!pip install pypdf
# !pip install chromadb

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install -Uq chromadb numpy datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


In [ ]:
# !pip uninstall numpy

In [ ]:
!pip install numpy==1.22

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.1 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is incompatible.
plotnine 0.12.2 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.


In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = [PyPDFLoader("rohit.pdf")]
docs = []
for load in loader:
  docs.extend(load.load())

In [ ]:
docs[0].page_content

"Rohit KumarINTERNSHIP EXPERIENCE EDUCATIONSarala Birla University  Dec 2021 - May 2025 B.Tech in Artificial Intelligence Senior Secondary Education XII : 92.2% SKILLS Technical Skills: +91-9798494187 rohitcode005@gmail.com Ranchi , Jharkhand Data Science: Pandas, NumPy, Scikit-learn\nDeep Learning: TensorFlow, OpenCV, LLM,\nCNN, RNN , Langchain\nDevelopment: Android, React, Flask\nAdditional: Deployment, MySQL, Postman,\nBeautiful Soup, PowerBI, Firebase, Git &\nGitHubProgramming Language: Python , C, C++ , Java , oops , Javascript ,\nHTMLData Engineer Taiyō.AI, Modern Intelligence for Global Infrastructure Jun 2021 -  Sep 2021 Perform web scraping of data from the World Bank and IADB websites,\ncreate a data pipeline to upload it to Google Cloud, and write Python\nscripts using Object-Oriented Programming (OOP) principles for data\npreprocessing with Pandas and NumPy.Data AnnotatorMarch 2023 - Jun 2023 KoiReader PROJECTS Speech Emotion Recognition Predict user emotions from audio inp

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
!pip install chromadb
!pip install pydantic-settings

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install tiktoken

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": "what is the collage of rohit"})
result["result"]